In [3]:
!pip install hypertools

In [5]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../timecorr/'))
from timecorr_main import timepoint_decoder
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0,0.1,1,5,10]
repetitions=100
noise_num = len(noise_level)
time_range=time_len = block_length * covariance_num
activation_num = 6
subject_num = 5
variance = 1000
activations = np.random.normal(0,1,[noise_num, subject_num, activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])


def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map1,feature_map1.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map2 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map2,feature_map2.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len/2):
        cov_temp = (time_len-i)*0.5*(np.log(1+correlation1+1e-5) - np.log(1-correlation1+1e-5))/float(time_len)+i*0.5*(np.log(1+correlation2+1e-5) - np.log(1-correlation2+1e-5))/float(time_len)
        correlations[i] =  (np.exp(2*cov_temp) - 1)/(np.exp(2*cov_temp) + 1)
        activations_temp[:,i] = np.dot(cholesky(correlations[i]),activations_temp[:,i])
    for i in range(time_len/2):
        correlations[time_len/2+i] =  correlations[time_len/2-i-1]
        activations_temp[:,time_len/2+i] = np.dot(cholesky(correlations[time_len/2+i]),activations_temp[:,time_len/2+i])
    for i in range(noise_num):
        activations[i]=np.tile(activations_temp,[subject_num,1,1] )+np.random.normal(0,noise_level[i],[subject_num, activation_num, time_len])
for i in range(noise_num):
    print(timepoint_decoder(activations[i],1000,3))

{'accuracy': 0.0, 'rank': 0.5, 'error': 0.33333333333333331}
{'accuracy': 0.16666666666666666, 'rank': 0.55555555555555547, 'error': 0.3666666666666667}
{'accuracy': 0.16666666666666666, 'rank': 0.63888888888888895, 'error': 0.5}
{'accuracy': 0.1111111111111111, 'rank': 0.56481481481481477, 'error': 0.39999999999999997}
{'accuracy': 0.16666666666666666, 'rank': 0.58333333333333337, 'error': 0.29999999999999999}
{'accuracy': 0.16666666666666666, 'rank': 0.63888888888888895, 'error': 0.3666666666666667}
{'accuracy': 0.16666666666666666, 'rank': 0.58333333333333337, 'error': 0.5}
{'accuracy': 0.16666666666666666, 'rank': 0.60185185185185197, 'error': 0.3888888888888889}
{'accuracy': 0.16666666666666666, 'rank': 0.55555555555555558, 'error': 0.3666666666666667}
{'accuracy': 0.16666666666666666, 'rank': 0.55555555555555558, 'error': 0.3666666666666667}
{'accuracy': 0.16666666666666666, 'rank': 0.61111111111111105, 'error': 0.40000000000000008}
{'accuracy': 0.16666666666666666, 'rank': 0.574